<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/Listening_Cloze_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio

In [ ]:
%%capture
# First, install the gTTS library if you haven't already
!pip install gTTS

# Import the gTTS library
from gtts import gTTS

In [ ]:
import gradio as gr
import requests
from gtts import gTTS
import os

# Cloze questions
cloze_questions = [
    {
        "question": "In the small mountain village of Echo Ridge, adventure ______ (be) a part of everyday life.",
        "answer": "was"
    },
    {
        "question": "Among these children ______ (be) a bright-eyed eighth grader named Alex.",
        "answer": "was"
    },
    {
        "question": "Alex ______ (know) for his daring spirit and his love for exploring the rugged landscapes around Echo Ridge.",
        "answer": "was known"
    },
    {
        "question": "He ______ (have) a particular fascination with the old maps and tales of hidden treasures that had been lost in the mountains centuries ago.",
        "answer": "had"
    },
    {
        "question": "One day, while exploring the local library, Alex ______ (stumble) upon an ancient map tucked inside a forgotten book on village lore.",
        "answer": "stumbled"
    },
    {
        "question": "The map ______ (hint) at the location of a lost treasure, hidden deep within a cave known as Whispering Hollow.",
        "answer": "hinted"
    },
]

# Function to convert text to speech and save as an audio file
def text_to_speech(text, filename):
    tts = gTTS(text)
    tts.save(filename)
    return filename

# Generate audio files for questions
for i, question in enumerate(cloze_questions):
    audio_filename = f"question_{i+1}.mp3"
    question["audio"] = text_to_speech(question["question"], audio_filename)

# Function to handle the cloze quiz
def cloze_quiz(state, answer):
    name, question_index, score, results = state
    question = cloze_questions[question_index]

    if answer.strip().lower() == question["answer"].lower():
        score += 1
        results.append(f"Question {question_index + 1}: Correct\n")
    else:
        results.append(f"Question {question_index + 1}: Incorrect, the correct answer is: {question['answer']}\n")

    question_index += 1

    if question_index < len(cloze_questions):
        next_question_audio = cloze_questions[question_index]["audio"]
        return (name, question_index, score, results), next_question_audio, gr.update(visible=False), gr.update(value="")
    else:
        result_text = f"* Name: {name}\n* Score: {score} out of {len(cloze_questions)}\n" + "\n".join(results)
        return (name, question_index, score, results), "", gr.update(visible=True, value=result_text), gr.update(visible=False)

# Function to start the quiz
def start_quiz(name):
    return (name, 0, 0, []), cloze_questions[0]["audio"], gr.update(visible=False), gr.update(visible=True)

# Create the Gradio interface
with gr.Blocks() as iface:
    name_input = gr.Textbox(label="Enter your name")
    start_button = gr.Button("Start Quiz")
    question_audio = gr.Audio(interactive=False)
    answer_input = gr.Textbox(label="Your Answer")
    next_button = gr.Button("Next")
    result_output = gr.Textbox(label="Results", interactive=False, visible=False)

    # Initialize the state
    state = gr.State()

    start_button.click(start_quiz, inputs=name_input, outputs=[state, question_audio, result_output, answer_input])
    next_button.click(cloze_quiz, inputs=[state, answer_input], outputs=[state, question_audio, result_output, answer_input])

iface.launch(share=True)
